In [18]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType

In [2]:
spark_session = SparkSession.builder.appName('Min temperature for each stations').getOrCreate()

In [4]:
schema = StructType([
    StructField(name='station_id', dataType=StringType(), nullable=True),
    StructField(name='date', dataType=IntegerType(), nullable=True),
    StructField(name='measure_type', dataType=StringType(), nullable=True),
    StructField(name='temperature', dataType=FloatType(), nullable=True)
])

In [12]:
data = spark_session.read.schema(schema).csv('../../data/1800.csv')
data.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: float (nullable = true)



In [20]:
min_temps = data.filter(data.measure_type == 'TMIN')\
                .select('station_id', 'temperature')\
                .groupBy('station_id') \
                .agg(min('temperature').alias('min_temp')) \
                .select('station_id', 'min_temp')
min_temps.show(5)

+-----------+--------+
| station_id|min_temp|
+-----------+--------+
|ITE00100554|  -148.0|
|EZE00100082|  -135.0|
+-----------+--------+



In [21]:
spark_session.stop()